In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import datetime
import os
import seaborn as sns
from datetime import date
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [22]:
xtrain = pd.read_csv("../data/Airbnb_xtrain_Version2.csv", index_col=0)
ytrain = pd.read_csv("../data/Airbnb_ytrainVersion2.csv",index_col=0)

In [23]:
#Checking null values
xtrain.isnull().sum()
xtrain = xtrain.dropna()

In [24]:
#Checking null values
ytrain.isnull().sum()
ytrain = ytrain.dropna()

In [25]:
xtrain

,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,...,c_447,c_448,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456
language,,,,,,,,,,,,,,,,,,,,,
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,11.0,27.0,15.0,4.0,9.0,10.0,14.0,5.0,1.0,0.0
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,16.0,22.0,28.0,29.0,7.0,6.0,7.0,7.0,1.0,0.0
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
en,2014.0,6.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3.0,6.0,1.0,1.0,0.0,1.0,0.0,0.0
en,2014.0,6.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,11.0,24.0,7.0,12.0,8.0,5.0,0.0,1.0,0.0
en,2014.0,6.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,31.0,21.0,15.0,12.0,9.0,4.0,5.0,2.0,1.0


In [26]:
ytrain

""
country_destination
other
NDF
NDF
NDF
GB
...
US
NDF
US


In [27]:
xtrain_new = xtrain.values
ytrain_new = ytrain.values

In [28]:
from sklearn.metrics import make_scorer

def dcg_score(y_true, y_score, k=5):

    """
    y_true : array, shape = [n_samples] 
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
    """
    order = np.argsort(y_score)[::-1] # Sort the score from high to low
    y_true = np.take(y_true, order[:k]) # Get top k score

    gain = 2 ** y_true - 1   

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

def ndcg_score(ground_truth, predictions, k=5):   

    """
    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes] 
        Predicted probabilities. 
    k : int
        Rank.
    """
    lb = LabelBinarizer()
    lb.fit(range(len(predictions) + 1))
    T = lb.transform(ground_truth)    
    scores = []
    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)


In [29]:
from sklearn.svm import SVC, LinearSVC, NuSVC
TOL = 1e-4
MAX_ITER = 1000
svm_name = []
RANDOM_STATE=1000
clf_svm = {

    'SVM-rbf': SVC(kernel='rbf',
                   max_iter=MAX_ITER,
                   tol=TOL, random_state=RANDOM_STATE,
                   decision_function_shape='ovr'),     

    'SVM-poly': SVC(kernel='poly',
                   max_iter=MAX_ITER,
                   tol=TOL, random_state=RANDOM_STATE,
                   decision_function_shape='ovr'),     

    'SVM-linear': SVC(kernel='linear',
                      max_iter=MAX_ITER,
                      tol=TOL, 
                      random_state=RANDOM_STATE,
                      decision_function_shape='ovr'),  

    'LinearSVC': LinearSVC(max_iter=MAX_ITER,
                            tol=TOL,
                            random_state=RANDOM_STATE,
                            multi_class = 'ovr') 
    }

train_score_svm = []
cv_score_svm = []

kf = KFold(n_splits=5,shuffle=True ,random_state=RANDOM_STATE)

k_ndcg = 5

for key in clf_svm.keys():

    clf = clf_svm.get(key)

    train_score_iter = []
    cv_score_iter = []

    for train_index, test_index in kf.split(xtrain_new, ytrain_new):

        X_train, X_test = xtrain_new[train_index, :], xtrain_new[test_index, :]
        y_train, y_test = ytrain_new[train_index], ytrain_new[test_index]

        clf.fit(X_train, y_train)

        y_pred = clf.decision_function(X_test)
        train_ndcg_score = ndcg_score(y_train, clf.decision_function(X_train), k = k_ndcg)
        cv_ndcg_score = ndcg_score(y_test, y_pred, k=k_ndcg)

        train_score_iter.append(train_ndcg_score)
        cv_score_iter.append(cv_ndcg_score)

    train_score_svm.append(np.mean(train_score_iter))
    cv_score_svm.append(np.mean(cv_score_iter))
    svm_name.append(key)

ValueError: Found array with 0 feature(s) (shape=(59049, 0)) while a minimum of 1 is required.